In [5]:
import tkinter as tk
from tkinter import messagebox

# 15 emergency kit items: (weight in kg - x.y where y is even), value
ITEMS = {
    "Water Bottle": (1.0, 10),
    "First Aid Kit": (2.0, 15),
    "Flashlight": (1.2, 8),
    "Multi-tool": (0.6, 12),
    "Emergency Blanket": (0.8, 9),
    "Food Ration": (1.4, 11),
    "Power Bank": (0.4, 10),
    "Raincoat": (1.2, 7),
    "Map": (0.2, 4),
    "Whistle": (0.2, 5),
    "Portable Stove": (1.6, 14),
    "Rope": (1.0, 6),
    "Gloves": (0.6, 5),
    "Compass": (0.4, 6),
    "Insect Repellent": (0.6, 7)
}

class KnapsackApp:
    def __init__(self, master):
        self.master = master
        master.title("Emergency Kit Knapsack Solver")

        tk.Label(master, text="Enter Weight Limit (kg):").grid(row=0, column=0, sticky="w")
        self.weight_limit_entry = tk.Entry(master)
        self.weight_limit_entry.grid(row=0, column=1, sticky="w")

        tk.Label(master, text="Select up to 7 items (no repeats):").grid(row=1, column=0, columnspan=3, sticky="w")

        self.item_vars = []
        self.dropdowns = []
        self.info_labels = []

        for i in range(7):
            var = tk.StringVar(master)
            var.set("None")
            self.item_vars.append(var)

            dropdown = tk.OptionMenu(master, var, "None", *ITEMS.keys(), command=lambda _, i=i: self.update_dropdowns(i))
            dropdown.grid(row=i+2, column=0, columnspan=2, sticky="we")
            self.dropdowns.append(dropdown)

            info_label = tk.Label(master, text="", width=30, anchor="w")
            info_label.grid(row=i+2, column=2, sticky="w")
            self.info_labels.append(info_label)

        self.solve_button = tk.Button(master, text="Solve Knapsack", command=self.solve_knapsack)
        self.solve_button.grid(row=10, column=0, columnspan=3, pady=(10, 5))

        self.result_text = tk.Text(master, height=30)
        self.result_text.grid(row=11, column=0, columnspan=3, pady=10)

    def get_selected_items(self):
        return [var.get() for var in self.item_vars if var.get() != "None"]

    def update_dropdowns(self, changed_index):
        selected_items = self.get_selected_items()

        # Update dropdowns to prevent duplicates
        for i, dropdown in enumerate(self.dropdowns):
            menu = dropdown["menu"]
            current_value = self.item_vars[i].get()
            menu.delete(0, "end")
            for item in ["None"] + list(ITEMS.keys()):
                if item == current_value or (item != "None" and item not in selected_items):
                    menu.add_command(label=item, command=lambda v=item, idx=i: self.set_item(idx, v))

        # Update info labels
        for i, var in enumerate(self.item_vars):
            item = var.get()
            if item in ITEMS:
                weight, value = ITEMS[item]
                self.info_labels[i].config(text=f"Weight: {weight} kg | Value: {value}")
            else:
                self.info_labels[i].config(text="")

    def set_item(self, index, value):
        self.item_vars[index].set(value)
        self.update_dropdowns(index)

    def solve_knapsack(self):
        try:
            weight_limit = float(self.weight_limit_entry.get())
            if weight_limit <= 0:
                raise ValueError
        except ValueError:
            messagebox.showerror("Input Error", "Please enter a valid positive number for weight limit.")
            return

        selected_items = self.get_selected_items()
        if not selected_items:
            messagebox.showinfo("No Items", "Please select at least one item.")
            return

        weights = [ITEMS[item][0] for item in selected_items]
        values = [ITEMS[item][1] for item in selected_items]
        n = len(selected_items)

        # Use 0.2 kg resolution -> Multiply by 5
        W = int(weight_limit * 5)
        int_weights = [int(w * 5) for w in weights]

        # Initialize DP Table
        dp = [[0] * (W + 1) for _ in range(n + 1)]

        for i in range(1, n + 1):
            for w in range(W + 1):
                if int_weights[i-1] <= w:
                    dp[i][w] = max(dp[i-1][w], values[i-1] + dp[i-1][w - int_weights[i-1]])
                else:
                    dp[i][w] = dp[i-1][w]

        # Backtrack for selected items
        res_weight = W
        selected = []
        for i in range(n, 0, -1):
            if dp[i][res_weight] != dp[i-1][res_weight]:
                selected.append(selected_items[i-1])
                res_weight -= int_weights[i-1]

        total_weight = sum(ITEMS[item][0] for item in selected)
        total_value = dp[n][W]

        # Adjust GUI width based on DP table
        dp_table_width = (W + 1) * 4 + 6
        text_width = min(dp_table_width, 300)
        self.result_text.config(width=text_width)
        window_width_px = min(dp_table_width * 5, 1800)
        self.master.geometry(f"{window_width_px}x700")

        # Display result
        self.result_text.delete("1.0", tk.END)
        self.result_text.insert(tk.END, "✅ Optimal Items Selected:\n")
        for item in selected:
            self.result_text.insert(tk.END, f"- {item} (Weight: {ITEMS[item][0]} kg, Value: {ITEMS[item][1]})\n")
        self.result_text.insert(tk.END, f"\n📦 Total Weight: {total_weight:.1f} kg\n")
        self.result_text.insert(tk.END, f"💰 Total Value: {total_value}\n\n")

        # DP Table Steps with 0.2 kg headers
        self.result_text.insert(tk.END, "📊 DP Table Steps (rows = items, cols = weight in 0.2 kg units):\n")
        header = "     " + "".join(f"{w*0.2:4.1f}" for w in range(W + 1)) + "\n"
        self.result_text.insert(tk.END, header)

        for i in range(n + 1):
            row = f"I{i:2} " + "".join(f"{dp[i][w]:4}" for w in range(W + 1)) + "\n"
            self.result_text.insert(tk.END, row)

# Run the GUI
root = tk.Tk()
app = KnapsackApp(root)
root.mainloop()
